### AMN-Wt on *E. coli* core FBA simulated training set
This code help us to change the code of the project. It allows us to check that nothing changes while refactoring.



In [1]:
import os
import time
import numpy as np
import tensorflow as tf
from aMNWtModel import AMNWtModel
from tools import printout

DIRECTORY = './'
SAVE_RESERVOIR = False

seed = 10
np.random.seed(seed=seed)  
tf.random.set_seed(seed)


## (not working with M1 chips ). I don't understand :)
# Create, train and evaluate AMN_Wt models with FBA simulated training set for E. coli core with upper bound (UB) or exact bound (EB) 
train_name = 'e_coli_core_UB' # e_coli_core_EB
reservoir_name = train_name + "_AMN_Wt"
training_file = os.path.join(DIRECTORY,'Dataset_model/',train_name)

print("---------------------------------------- model ----------------------------------------")
model = AMNWtModel(training_file = training_file, 
                   objective=['BIOMASS_Ecoli_core_w_GAM'],  
                   model_type='AMN_Wt', 
                   timestep =4,
                   n_hidden = 1,
                   hidden_dim = 50,
                   scaler=True,
                   train_rate=1e-2,
                   epochs=20, 
                   xfold=5,
                   verbose=True,
                   batch_size=7)

model.train_test_split(test_size=0.1, random_state=seed)
model.printout()

print("---------------------------------------- train and evaluate ----------------------------------------")
start_time = time.time()
_, stats, _ = model.train_evaluate(verbose=False)
reservoir = model
delta_time = time.time() - start_time

print("---------------------------------------- printing cross-validation results ----------------------------------------")
stats.printout(reservoir_name, delta_time)


print("---------------------------------------- evaluate model on test set ----------------------------------------")
if SAVE_RESERVOIR:
    reservoir_file = os.path.join(DIRECTORY,'Reservoir/',reservoir_name)
    reservoir.save(reservoir_file)

reservoir.printout()

start_time = time.time()

## Strange two first lines, investigate
reservoir.X, reservoir.Y = model.X_test, model.Y_test
X, Y = reservoir.model_input(model.X_test, model.Y_test, verbose=False)
pred, obj, loss = reservoir.evaluate_model(X, Y, verbose=False)
delta_time = time.time() - start_time
printout('Test set', delta_time, obj, loss)

---------------------------------------- model ----------------------------------------
number of reactions:  154 154
number of metabolites:  72
filtered measurements size:  1
training file: ./Dataset_model/e_coli_core_UB
model type: AMN_Wt
model scaler: 1.0
model input dim: 0
model output dim: 0
model medium bound: UB
timestep: 4
training set size (1000, 20) (1000, 1)
nbr hidden layer: 1
hidden layer size: 50
activation function: relu
training epochs: 20
training regression: True
training learn rate: 0.01
training droP_out: 0.25
training batch size: 7
training validation iter: 0
training xfold: 5
training early stopping: False
---------------------------------------- train and evaluate ----------------------------------------
Instructions for updating:
Use `tf.linalg.matmul` instead


2023-04-20 14:43:30.185082: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-20 14:43:30.186659: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-04-20 14:43:30.645454: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


train = 0.95 test = 0.96 loss-train = 0.004603 loss-test = 0.004677
train = 0.96 test = 0.96 loss-train = 0.004758 loss-test = 0.004760
train = 0.94 test = 0.93 loss-train = 0.004356 loss-test = 0.004264
train = 0.95 test = 0.94 loss-train = 0.004296 loss-test = 0.004455
train = 0.97 test = 0.96 loss-train = 0.004050 loss-test = 0.003941
---------------------------------------- printing cross-validation results ----------------------------------------
Stats for e_coli_core_UB_AMN_Wt CPU-time 174.5725
R2 = 0.9514 (+/- 0.0108) Constraint = 0.0044 (+/- 0.0002)
Q2 = 0.9509 (+/- 0.0114) Constraint = 0.0044 (+/- 0.0003)
---------------------------------------- evaluate model on test set ----------------------------------------
training file: ./Dataset_model/e_coli_core_UB
model type: AMN_Wt
model scaler: 10.0
model input dim: 4
model output dim: 0
model medium bound: UB
timestep: 4
training set size (1000, 20) (1000, 1)
nbr hidden layer: 1
hidden layer size: 50
activation function: relu
trai